In [4]:
from scipy.optimize import minimize

loading data from the sql 

In [5]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# --- PostgreSQL Connection ---
engine = create_engine('postgresql://akilfiros:@127.0.0.1:5432/postgres')

# --- Load historical price data ---
query = "SELECT date, ticker, close FROM financial_data"
df = pd.read_sql(query, engine)
df['date'] = pd.to_datetime(df['date'])

# --- Pivot to get price matrix [date x ticker] ---
price_df = df.pivot(index='date', columns='ticker', values='close').sort_index()
price_df = price_df.dropna(axis=1)  # Drop tickers with missing data

# --- Compute log returns ---
log_returns = np.log(price_df / price_df.shift(1)).dropna()

# ✅ Preview
print("Log returns shape:", log_returns.shape)
print(log_returns.head())


Log returns shape: (1256, 92)
ticker          AAPL      ADBE       ADI       ADP       AEP      ALGN  \
date                                                                     
2020-01-03 -0.009770 -0.007865 -0.017760 -0.002116 -0.001071 -0.011487   
2020-01-06  0.007937  0.005710 -0.011818  0.001352  0.003315  0.019212   
2020-01-07 -0.004714 -0.000959  0.022496 -0.012178  0.000213 -0.009913   
2020-01-08  0.015959  0.013348  0.008991  0.009354 -0.002993  0.010333   
2020-01-09  0.021018  0.007607  0.000000  0.008800  0.005019  0.036190   

ticker          AMAT       AMD      AMGN      AMZN  ...      TEAM      TSLA  \
date                                                ...                       
2020-01-03 -0.016044 -0.010236 -0.006812 -0.012213  ...  0.001301  0.029203   
2020-01-06 -0.021801 -0.004330  0.007645  0.014776  ...  0.037096  0.019072   
2020-01-07  0.028477 -0.002897 -0.009450  0.002089  ...  0.003596  0.038067   
2020-01-08 -0.000649 -0.008743  0.000756 -0.007839  ... 

calculating expected returns & covariance matirx 

In [6]:
# Assuming 252 trading days per year
expected_returns = log_returns.mean() * 252

# --- Compute annualized covariance matrix ---
cov_matrix = log_returns.cov() * 252

# --- Preview ---
print("✅ Expected Returns (annualized):")
print(expected_returns.head())

print("\n✅ Covariance Matrix (annualized):")
print(cov_matrix.iloc[:5, :5])  # Show a 5x5 slice


✅ Expected Returns (annualized):
ticker
AAPL    0.249304
ADBE    0.057672
ADI     0.132328
ADP     0.130070
AEP     0.033292
dtype: float64

✅ Covariance Matrix (annualized):
ticker      AAPL      ADBE       ADI       ADP       AEP
ticker                                                  
AAPL    0.100265  0.076782  0.067680  0.047995  0.027556
ADBE    0.076782  0.153592  0.079548  0.053579  0.021740
ADI     0.067680  0.079548  0.126015  0.054358  0.023100
ADP     0.047995  0.053579  0.054358  0.077391  0.034303
AEP     0.027556  0.021740  0.023100  0.034303  0.061746


In [7]:
# --- Prepare data ---
tickers = expected_returns.index.tolist()
mu = expected_returns.values
cov = cov_matrix.values
n_assets = len(tickers)

# --- Objective function: portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested ---
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# --- Bounds: long-only portfolio (0 <= w <= 1) ---
bounds = [(0.0, 1.0) for _ in range(n_assets)]

# --- Initial guess: equal weighting ---
w0 = np.ones(n_assets) / n_assets

# --- Solve optimization ---
result = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                  bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights = pd.Series(result.x, index=tickers)

# --- Display result ---
print("✅ Optimal Weights (Minimum Variance Portfolio):")
print(optimal_weights[optimal_weights > 0].sort_values(ascending=False))


✅ Optimal Weights (Minimum Variance Portfolio):
KDP     1.770915e-01
GILD    1.425197e-01
AEP     1.192630e-01
EA      1.175635e-01
COST    1.065420e-01
AZN     1.054925e-01
PCAR    5.522733e-02
MAR     3.379642e-02
REGN    3.274592e-02
KHC     2.954738e-02
TCOM    1.952500e-02
NTES    1.892534e-02
TTWO    1.112001e-02
MDLZ    1.091417e-02
ORLY    8.917609e-03
DLTR    4.757982e-03
PANW    3.900272e-03
AMGN    2.150375e-03
ADP     5.991958e-17
CDNS    5.212991e-17
EXC     3.546428e-17
HON     2.867561e-17
MTCH    2.746492e-17
ADI     2.164018e-17
BIDU    2.111682e-17
LULU    2.042658e-17
AMZN    1.885267e-17
BKNG    1.734248e-17
PAYX    1.616112e-17
TSLA    1.505869e-17
ZS      1.302769e-17
BKR     1.251199e-17
NVDA    1.201862e-17
WBA     1.033459e-17
MCHP    1.023706e-17
IDXX    9.878210e-18
CSCO    9.565955e-18
ALGN    9.226053e-18
NFLX    6.688135e-18
INTC    6.648125e-18
ISRG    6.445968e-18
OKTA    6.049744e-18
ANSS    5.827621e-18
AXON    5.754006e-18
MSFT    5.134822e-18
PDD    


- These are the optimal asset weights (allocations) that minimize the total portfolio variance, assuming:
    - Full investment (sum = 1)
    - No short selling (weights ≥ 0)
- Assets with very small values (e.g., 1e-17) are essentially excluded from the portfolio — their weight is effectively zero.

Calculating for Constrained Portfolio Optimization (Target Return + Max Weight)

In [8]:
# --- Target return: e.g., at least 5% annual return ---
target_return = 0.05

# --- Objective: minimize portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested + target return ---
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
    {'type': 'ineq', 'fun': lambda w: w @ mu - target_return}  # return constraint
]

# --- Bounds: long-only + max 10% in any asset ---
max_weight = 0.10
bounds = [(0.0, max_weight) for _ in range(n_assets)]

# --- Initial guess ---
w0 = np.ones(n_assets) / n_assets

# --- Solve ---
result_constrained = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                              bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights_constrained = pd.Series(result_constrained.x, index=tickers)

# --- Display non-zero weights ---
print("✅ Constrained Optimal Weights:")
print(optimal_weights_constrained[optimal_weights_constrained > 0].sort_values(ascending=False))

✅ Constrained Optimal Weights:
EA      1.000000e-01
AZN     1.000000e-01
KDP     1.000000e-01
AEP     1.000000e-01
GILD    1.000000e-01
            ...     
ON      1.136797e-18
MRVL    1.088411e-18
ROST    9.850348e-19
FTNT    2.056117e-19
MSTR    8.168223e-20
Length: 63, dtype: float64



- Enforced param:
    - Target return ≥ 5%
    - Max weight = 10% per asset
    - Long-only constraint (no negative weights)
- As a result:
    - Assets like EA, AZN, KDP, AEP, and GILD hit the maximum allowed allocation of 10% — they are likely low-risk, high-return contributors.
    - Several other assets have near-zero weights (e.g., MRVL, MSTR) — the optimizer deems them suboptimal under your constraints.


Saving connstrained weights to the sql 


In [9]:
from sqlalchemy import create_engine

# --- Prepare final weights table ---
weights_df = optimal_weights_constrained.reset_index()
weights_df.columns = ['ticker', 'weight']
weights_df = weights_df[weights_df['weight'] > 0].sort_values(by='weight', ascending=False)

# --- Save to PostgreSQL ---
engine = create_engine('postgresql://akilfiros:@127.0.0.1:5432/postgres')
weights_df.to_sql("optimal_portfolio_weights", engine, if_exists='replace', index=False)

print("✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights")

✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights
